In [176]:
import random

EMPTY = 100
AGENT = 200
FRUIT = 300
LEFTUP_HUDDLE = 77
LEFT_HUDDLE = 44
LEFTDOWN_HUDDLE = 11
DOWN_HUDDLE = 22
RIGHTDOWN_HUDDLE = 33
RIGHT_HUDDLE = 66
RIGHTUP_HUDDLE = 99
UP_HUDDLE = 88
MIDDLE_HUDDLE = 55

LEFT = 0
UP = 1
RIGHT = 2
DOWN = 3
    
DISCOUNT_RATE = 0.95

WIDTH = 10
HEIGHT = 8

In [177]:
class Agent:
        
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def setX(self, x):
        self.x = x
 
    def setY(self, y):
        self.y = y
             
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    
    def move_proba(self, policy):
        '''
        확률에 따라 움직이는 함수(학습 시 사용)
        Agent가 움직일 방향의 정책이 음수라면 해당 방향의 Action은 고려하지 않음.
        Agent가 움직일 방향의 정책에 양수가 있다면 가장 큰 쪽으로 움직임.
        Agent가 움직일 방향의 max값이 0이라면 0중에 랜덤한 방향으로 이동함.
        '''
        
        # hist는 Agent가 Action 하기 전 위치를 기억하기 위한 변수
        # Action을 취한 그리드가 보상인지 벌인지에 따라 해당 위치에 가치를 매겨줘야 함.
        self.hist = [self.getX(), self.getY()]
        
        if max(policy[self.getY()][self.getX()]) == 0:
            direction = random.choice([i for i in range(len(policy[self.getY()][self.getX()])) if policy[self.getY()][self.getX()][i] == 0])
        elif max(policy[self.getY()][self.getX()]) > 0:
            direction = policy[self.getY()][self.getX()].index(max(policy[self.getY()][self.getX()]))
        
        if direction == LEFT:
            self.setX(self.getX()-1)
        elif direction == UP:
            self.setY(self.getY()-1)
        elif direction == RIGHT:
            self.setX(self.getX()+1)
        elif direction == DOWN:
            self.setY(self.getY()+1)
        
        self.hist.append(direction)
    
    
    def move_argmax(self, policy):
        '''
        학습을 마친 후, 가장 가치가 가장 큰 쪽으로만 움직이기 위한 함수.
        '''
        
        self.hist = [self.getX(), self.getY()]
        
        direction = policy[self.getY()][self.getX()].index(max(policy[self.getY()][self.getX()]))
        
        if direction == LEFT:
            self.setX(self.getX()-1)
        elif direction == UP:
            self.setY(self.getY()-1)
        elif direction == RIGHT:
            self.setX(self.getX()+1)
        elif direction == DOWN:
            self.setY(self.getY()+1)
            

In [180]:
class Environment:
        
    def __init__(self, w, h, fruit_x, fruit_y, agent, policy):
        self.width = w
        self.height = h
        self.fruit_x = fruit_x
        self.fruit_y = fruit_y
        self.agent = agent
        self.score = 0
        
        # 맵을 그리기위한 _map변수.
        # 시각화 하는 쪽이 이해하기나 디버깅하기 편하기 때문...
        # 나는 벽에 닿으면 죽는 것 까지 고려하기 위해 Width, Height에 +2 를 해서 벽의 공간을 만듦.
        self._map = [[EMPTY for x in range(self.width+2)] for y in range(self.height+2)]
        self._map[self.agent.getY()][self.agent.getX()] = AGENT
        self._map[self.fruit_y][self.fruit_x] = FRUIT
        for i in range(len(self._map)):
            for j in range(len(self._map[i])):
                if i == 0 and j == 0:
                    self._map[i][j] = LEFTUP_HUDDLE
                elif i == 0 and j == len(self._map[i])-1:
                    self._map[i][j] = RIGHTUP_HUDDLE
                elif i == len(self._map)-1 and j == len(self._map[i])-1:
                    self._map[i][j] = RIGHTDOWN_HUDDLE
                elif i == len(self._map)-1 and j == 0:
                    self._map[i][j] = LEFTDOWN_HUDDLE
                elif i == 0:
                    self._map[i][j] = UP_HUDDLE
                elif j == 0:
                    self._map[i][j] = LEFT_HUDDLE
                elif i == len(self._map)-1:
                    self._map[i][j] = DOWN_HUDDLE
                elif j == len(self._map[i])-1:
                    self._map[i][j] = RIGHT_HUDDLE
                    
        self.policy = policy
        
        # 과일을 먹는다면 보상으로 +10점
        # 벽이나 장애물에 닿는다면 벌점으로 -1점
        self.reward = [[0 for x in range(self.width+2)] for y in range(self.height+2)]
        for i in range(len(self.reward)):
            for j in range(len(self.reward[i])):
                if self._map[i][j] == LEFTUP_HUDDLE or self._map[i][j] == LEFT_HUDDLE or self._map[i][j] == LEFTDOWN_HUDDLE or self._map[i][j] == DOWN_HUDDLE or self._map[i][j] == RIGHTDOWN_HUDDLE or self._map[i][j] == RIGHT_HUDDLE or self._map[i][j] == RIGHTUP_HUDDLE or self._map[i][j] == UP_HUDDLE or self._map[i][j] == MIDDLE_HUDDLE:
                    self.reward[i][j] = -1
        self.reward[self.fruit_y][self.fruit_x] = 10
        
        self.isEnd = False
        
        
    def getFruitXY(self):
        return [self.fruit_x, self.fruit_y]
    
    
    def updateAgent(self):
        '''
        현재 환경에서 움직인 Agent를 Update해주기 위한 함수
        과일을 먹는다면 100점이 오르고 게임이 끝나고(version.20200529, 향후 과일을 먹으면 다른 위치에서 과일이 나오게 할 예정)
        벽에 닿는다면 그대로 게임이 끝난다.
        '''
        if self._map[self.agent.getY()][self.agent.getX()] == LEFTUP_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == LEFT_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == LEFTDOWN_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == DOWN_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == RIGHTDOWN_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == RIGHT_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == RIGHTUP_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == UP_HUDDLE or self._map[self.agent.getY()][self.agent.getX()] == MIDDLE_HUDDLE:
            self.isEnd = True
        elif self.agent.getX() == self.fruit_x and self.agent.getY() == self.fruit_y:
            self.score = 100
            self.isEnd = True

        self._map[self.agent.hist[1]][self.agent.hist[0]] = EMPTY
        self._map[self.agent.getY()][self.agent.getX()] = AGENT

            
            
            
            
    def calcPolicy(self):
        '''
        정책을 업데이트 해주기 위한 함수
        해당 위치의 정책 = Agent가 움직일 칸의 보상 + max(Agent가 움직일 칸의 정책) * 할인율
        '''
        self.policy[self.agent.hist[1]][self.agent.hist[0]][self.agent.hist[2]] = self.reward[self.agent.getY()][self.agent.getX()] + (max(self.policy[self.agent.getY()][self.agent.getX()]) * DISCOUNT_RATE)
        
        
        
    def getPolicy(self):
        return self.policy
        
    
    def printMap(self):
        '''
        맵을 그려주기 위한 함수
        '''
        print('score : '+str(self.score))
        for i in range(self.height+2):
            for j in range(self.width+2):
                if self._map[i][j] == EMPTY:
                    print('□', end='')
                elif self._map[i][j] == MIDDLE_HUDDLE:
                    print('■', end='')
                elif self._map[i][j] == LEFTUP_HUDDLE:
                    print('┏', end='')
                elif self._map[i][j] == LEFT_HUDDLE:
                    print('┃', end='')
                elif self._map[i][j] == LEFTDOWN_HUDDLE:
                    print('┗', end='')
                elif self._map[i][j] == DOWN_HUDDLE:
                    print('━', end='')
                elif self._map[i][j] == RIGHTDOWN_HUDDLE:
                    print('┛', end='')
                elif self._map[i][j] == RIGHT_HUDDLE:
                    print('┃', end='')
                elif self._map[i][j] == RIGHTUP_HUDDLE:
                    print('┓', end='')
                elif self._map[i][j] == UP_HUDDLE:
                    print('━', end='')
                elif self._map[i][j] == AGENT:
                    print('●', end = '')
                elif self._map[i][j] == FRUIT:
                    print('★', end ='')
            print()
        print('\n')
        
        
        
    def printPolicy(self):
        '''
        정책을 보여주기 위한 함수
        '''
        for i in self.policy:
            for j in i:
                print('[', end ='')
                for k in j:
                    print(round(k,2), end =',')
                print(']', end = ' ')
            print('\n')

    
    
        
    

In [181]:
# 특정위치에서의 학습

ITER = 200
cnt = 0

policy = [[[0,0,0,0] for x in range(WIDTH+2)] for y in range(HEIGHT+2)]

while cnt < ITER:
    print(cnt)
    cnt+=1
    
    agent = Agent(1,1)
    
    env = Environment(WIDTH, HEIGHT, 7, 7, agent, policy)
    #env.printMap()
    
    while not env.isEnd:
        agent.move_proba(env.getPolicy())
        env.updateAgent()
        env.calcPolicy()
        
        #env.printMap()
        
    #env.printPolicy()
    
    policy = env.getPolicy()
        


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [182]:
# 학습한 결과를 가지고 예측
agent = Agent(1,1)
    
env = Environment(WIDTH, HEIGHT, 7, 7, agent, policy)
env.printMap()
    
while not env.isEnd:
    agent.move_argmax(env.getPolicy())
    env.updateAgent()
        
    env.printMap()
        


score : 0
┏━━━━━━━━━━┓
┃●□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□★□□□┃
┃□□□□□□□□□□┃
┗━━━━━━━━━━┛


score : 0
┏━━━━━━━━━━┓
┃□●□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□★□□□┃
┃□□□□□□□□□□┃
┗━━━━━━━━━━┛


score : 0
┏━━━━━━━━━━┓
┃□□□□□□□□□□┃
┃□●□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□★□□□┃
┃□□□□□□□□□□┃
┗━━━━━━━━━━┛


score : 0
┏━━━━━━━━━━┓
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□●□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□★□□□┃
┃□□□□□□□□□□┃
┗━━━━━━━━━━┛


score : 0
┏━━━━━━━━━━┓
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□●□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□★□□□┃
┃□□□□□□□□□□┃
┗━━━━━━━━━━┛


score : 0
┏━━━━━━━━━━┓
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□●□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□★□□□┃
┃□□□□□□□□□□┃
┗━━━━━━━━━━┛


score : 0
┏━━━━━━━━━━┓
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□●□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□□□□□┃
┃□□□□□□★□□□┃
┃□□□□□□□□□□┃
┗━━━━━━━━━━┛


score 